# Kucoin Websocket

<ol>
    <li> Apply connect token, obtain a response
        <ul>
            <li> POST /Api/V1/bullet-public
                <li> POST /Api/V1/bullet-private
        </ul>
    <li> Create connection (with websocket)
        <ul>
            <li> If connection succesfully stablished you receive a welcome message
        </ul>
    <li> Set Ping
        <ul>
            <li> To keep alive the link, the clinet side needs to send ping messages every pingInterval
            <li> After ping is sent, the system would return a pong message to the client side.
        </ul>
    <li> Subscribe
        <ul>
            <li> Send subscription message to the server
        </ul>
</ol>

### Subscribe to topic
<code> #websocket response
{
    'code': '200000', 
    'data': {'token': '2neAiuYvAU61ZDXANAGAsiL4-iAExhsBXZxftpOeh_55i3Ysy2q2LEsEWU64mdzUOPusi34M_wGoSf7iNyEWJ7mWo7DW2HzV3eE5sq9oXsa_GYS3W2hx-9iYB9J6i9GjsxUuhPw3BlrzazF6ghq4L8Ib_mfJpXxYRWYg6LH3wio=.abmr_NDQl7YHlhDnCcksbA==', 
    'instanceServers': [
    {
    'endpoint': 'wss://ws-api.kucoin.com/endpoint', 
    'encrypt': True, 
    'protocol': 'websocket', 
    'pingInterval': 18000, 
    'pingTimeout': 10000}
                      ]
    }
}</code>

### Public channels

Topic: /market/ticker:{symbol},{symbol}...

<code>{
    "id": 1545910660739,
    "type": "subscribe",
    "topic": "/market/ticker:BTC-USDT",
    "response": true
}</code>

<code> {
    "id": 1545910660739,                          //The id should be an unique value
    "type": "subscribe",
    "topic": "/market/ticker:BTC-USDT,ETH-USDT",  //Topic needs to be subscribed. Some topics support to divisional subscribe the informations of multiple trading pairs through ",".
    "privateChannel": false,                      //Adopted the private channel or not. Set as false by default.
    "response": true                              //Whether the server needs to return the receipt information of this subscription or not. Set as false by default.
}</code>

In [ ]:
#imports

#import base64
#import hashlib
#import hmac
import json
import threading
import time
import traceback

import websocket
import requests

In [ ]:
def get_ws_token():
    url = 'https://api.kucoin.com/api/v1/bullet-public'
    #now = int(time.time() * 1000)
    #str_to_sign = str(now) + 'POST' + '/api/v1/bullet-public'
    #signature = base64.b64encode(
        #hmac.new(SECRET_KEY.encode('utf-8'), str_to_sign.encode('utf-8'), hashlib.sha256).digest())
    #headers = {
    #    "KC-API-SIGN": signature,
    #    "KC-API-TIMESTAMP": str(now),
    #    "KC-API-KEY": API_KEY,
    #    "KC-API-PASSPHRASE": PASS_PHRASE
    #}
    response = requests.request('post', url) #, headers=headers
    return response.json()['data']['token']
    #print(response.json()['data']['token'])

#if __name__ == "__main__":
#    get_ws_token()

In [ ]:
def on_message(wsapp, message):
    print(message)


def on_error(wsapp, error):
    print(error)
    
#wsapp = websocket.WebSocketApp("wss://stream.meetup.com/2/rsvps", on_message=on_message)
#wsapp.run_forever()

In [ ]:
def launch():

    while True:
        print('Websocket running')
        time.sleep(.1)

        token = get_ws_token()
        print(token)

        try:
            ws = websocket.WebSocketApp(
                f"wss://push1-v2.kucoin.com/endpoint?token={token}&connectId=1&acceptUserMessage=true",
                on_open=on_open,
                on_message=on_message,
                on_error=on_error,
                on_close=on_close)

            ws.run_forever()
        except:
            print('ws Error: websocket failed')
            print(traceback.format_exc())

            
if __name__ == '__main__':
    try:
        launch()
    except KeyboardInterrupt:
        exit()